In [190]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [191]:
import os
import json
import dotenv

dotenv.load_dotenv()

True

In [192]:
from typing import TypedDict, Annotated, List, Dict, Literal
from typing_extensions import TypedDict
from IPython.display import display, Markdown, Image
from datetime import datetime
import operator

In [193]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent

In [194]:
from langgraph.graph import START, END, StateGraph
from langgraph.types import Command, Send
from langchain_core.messages import HumanMessage, BaseMessage, SystemMessage
from langchain_experimental.utilities import PythonREPL
from langgraph.graph.message import add_messages

In [195]:
from src.utils_trans import search_flight_details

In [196]:
def get_system_prompt_tool(suffix: str) -> str:
    return (
        "You are a helpful AI assistant, collaborating with other assistants."
        " Use the provided tools to progress towards answering the question."
        " Always format your final answer as a JSON string."
        f"\n{suffix}"
    )

In [197]:
def get_system_prompt(suffix: str) -> str:
    return (
        "You are a helpful AI assistant, collaborating with other assistants."
        " Always format your final answer as a JSON string."
        f"\n{suffix}"
    )

In [198]:
llm = ChatDeepSeek(model="deepseek-chat",
                   api_key=os.getenv("DEEPSEEK_API_KEY"))

In [199]:
class State(TypedDict):
    messages: Annotated[List, add_messages]
    trip_start_date: str
    trip_end_date: str
    traveler_preferences: Dict
    start_flight: str
    return_flight: str
    

    flight_info: Dict
    total_schedule: Dict 
    report_markdown: str

In [200]:
@tool
def get_flight_data(start_flight: str, return_flight: str) -> str:
    """
    Retrieve flight details for given flight numbers.
    """
    result = search_flight_details(start_flight, return_flight)
    json.dumps(result)
    return json.dumps(result)

In [201]:
flight_data_agent = create_react_agent(
    llm,
    tools=[get_flight_data],
    prompt=get_system_prompt_tool(
        "You are a travel planning assistant specializing in retrieving round-trip flight information. "
        "Use the get_flight_data tool with provided flight numbers and return its output directly as a JSON string. "
        "Do NOT include any explanatory text, comments, or non-JSON content. "
        "Return an empty JSON object {} if no flight data is available or if the tool fails."
    ),
)

### Create Graph


In [202]:
# Helper: robustly extract JSON object from model output
import re

def _extract_json_object(text: str):
    if isinstance(text, dict):
        return text
    if not isinstance(text, str):
        raise json.JSONDecodeError("Non-string content", str(text), 0)
    # Strip code fences if present
    fenced = re.search(r"```(?:json)?\s*([\s\S]*?)```", text, re.IGNORECASE)
    if fenced:
        candidate = fenced.group(1).strip()
        try:
            return json.loads(candidate)
        except Exception:
            pass
    # Try greedy first/last brace
    first = text.find("{")
    last = text.rfind("}")
    if first != -1 and last != -1 and last > first:
        candidate = text[first:last+1]
        try:
            return json.loads(candidate)
        except Exception:
            pass
    # As-is
    return json.loads(text)


In [203]:
def flight_data_node(state: State) -> State:
    start_flight = state['start_flight']
    return_flight = state['return_flight']
    # Run agent
    agent_input = {"messages": state['messages'] + [HumanMessage(content=f"Get data for {start_flight} and {return_flight}")]}
    agent_output = flight_data_agent.invoke(agent_input)
    # Parse safely
    final_content = agent_output['messages'][-1].content
    try:
        if not final_content.strip():
            print("Warning: Agent output is empty")
            flight_info = {}
        else:
            flight_info = json.loads(final_content)
    except json.JSONDecodeError as e:
        print(f"JSON parsing failed: {e}\nContent was: {final_content}")
        flight_info = {}
    # Update state
    state['flight_info'] = flight_info
    state['messages'].append(SystemMessage(content=f"Flight info updated: {flight_info}"))
    return state

In [204]:
def calculate_trip_days(start_date: str, end_date: str) -> int:
    """Calculate the number of days in the trip, including start and end dates."""
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    return (end - start).days + 1

def _extract_json_object(text: str):
    import re
    import json

    # Try code-fence wrapped JSON
    if isinstance(text, dict):
        return text
    if not isinstance(text, str):
        raise json.JSONDecodeError("Non-string content", str(text), 0)

    fenced = re.search(r"```(?:json)?\s*([\s\S]*?)```", text, re.IGNORECASE)
    if fenced:
        candidate = fenced.group(1).strip()
        try:
            return json.loads(candidate)
        except Exception as e:
            print("Failed to parse fenced JSON:", e)
            print("Candidate was:", candidate[:500])  # Print first 500 chars for debug

    # Greedy braces fallback
    first = text.find("{")
    last = text.rfind("}")
    if first != -1 and last != -1 and last > first:
        candidate = text[first:last + 1]
        try:
            return json.loads(candidate)
        except Exception as e:
            print("Failed to parse greedy braces JSON:", e)
            print("Candidate was:", candidate[:500])

    # Direct parse
    try:
        return json.loads(text)
    except Exception as e:
        print("Failed to parse direct JSON:", e)
        print("Text was:", text[:500])
        raise


In [205]:
def daily_schedule_node(state: State):
    trip_start = state.get("trip_start_date")
    trip_end = state.get("trip_end_date")
    preferences = state.get("traveler_preferences", {})
    flight_info = state.get("flight_info", {})
    city = state.get("city", "")

    # Calculate number of trip days
    num_days = calculate_trip_days(trip_start, trip_end)

    # Build prompts that force a strict JSON-only response
    system_prompt = get_system_prompt(
        "Return ONLY a JSON object following required_output_format. Do not include any prose, code fences, or explanations."
    )

    # Updated user prompt with detailed instructions
    user_prompt = json.dumps(
        {
            "instructions": (
                "Generate a diverse activity pool for the specified city and travel dates. "
                "Food activities should include local restaurants, street food, or cooking classes. "
                "Culture activities should include museums, historical sites, or festivals. "
                "Light hiking should include scenic trails or parks suitable for light outdoor activity. "
                "Tailor activities to the city, travel dates (considering seasonal factors), and preferences. "
                "Schedule meals within lunch (12:00-13:00) and dinner (18:00-19:00) windows. "
                "If preferences are missing, include a balanced mix of food, culture, and light hiking. "
                "For multi-city trips, include transitions between cities based on flight_info. "
                "Each day should have a balanced schedule from 09:00 to 20:00, respecting meal times. "
                "On the first day, account for the start flight arrival time and limit activities to after arrival. "
                "On the last day, account for the return flight departure time and schedule activities before departure. "
                f"Generate a schedule for {num_days} days, labeling them as day1, day2, ..., day{num_days}. "
                "Each day should have 3-5 activities, including at least one meal (lunch or dinner) and one activity from each preference (food, culture, light hiking) where possible."
            ),
            "trip": {
                "city": city,
                "start_date": trip_start,
                "end_date": trip_end,
                "preferences": preferences,
            },
            "flight_info": flight_info,
            "assumptions": {
                "default_open": "09:00-20:00",
                "avg_transit_min": 20,
                "lunch_window": "12:00-13:00",
                "dinner_window": "18:00-19:00",
            },
            "required_output_format": {
                f"day{n}": {
                    "activities": [
                        {
                            "name": "string",
                            "location": "string",
                            "activity_type": "food|culture|light_hiking|meal|transit|rest",
                            "start_time": "HH:MM",
                            "end_time": "HH:MM",
                            "notes": "string"
                        }
                    ]
                } for n in range(1, num_days + 1)
            },
        }
    )
    
    messages = [SystemMessage(content=system_prompt), HumanMessage(content=user_prompt)]

    schedule_content =llm.invoke(messages).content
    try:
        parsed = _extract_json_object(schedule_content)
        state["total_schedule"] = parsed
    except Exception:
        state["total_schedule"] = {f"day{n}": {"activities": []} for n in range(1, num_days + 1)}
        state.setdefault("messages", []).append(SystemMessage(content="Failed to parse schedule JSON"))

    return state

In [206]:
def report_node(state):
    """
    Generate a markdown trip report from the new state format.
    """
    total = state.get("total_schedule", {})
    flight_info = state.get("flight_info", {})
    trip_start = state.get("trip_start_date", "")
    trip_end = state.get("trip_end_date", "")
    city = state.get("city", "")
    preferences = state.get("traveler_preferences", {})
    lines = []

    # Title and overview
    lines.append(f"# Trip Plan: {trip_start} to {trip_end}")
    lines.append("")
    lines.append("## Overview")
    lines.append(f"- **City:** {city}")
    if preferences:
        lines.append(f"- **Preferences:** {preferences}")
    if flight_info:
        lines.append("- **Flights:**")
        start_f = flight_info.get("start_flight", {})
        ret_f = flight_info.get("return_flight", {})
        if start_f:
            lines.append(
                f"  - Outbound: {start_f.get('departure_airport','')} → {start_f.get('arrival_airport','')} "
                f"({start_f.get('departure_time','')}–{start_f.get('estimated_arrival_time','')})"
            )
        if ret_f:
            lines.append(
                f"  - Return: {ret_f.get('departure_airport','')} → {ret_f.get('arrival_airport','')} "
                f"({ret_f.get('departure_time','')}–{ret_f.get('estimated_arrival_time','')})"
            )
    lines.append("")

    # Daily schedule
    lines.append("## Daily Schedule")
    for day_key in sorted(total.keys(), key=lambda k: int(k[3:]) if k.startswith('day') and k[3:].isdigit() else k):
        day = total[day_key]
        lines.append(f"### {day_key.capitalize()}")
        lines.append("| Time | Type | Name | Location | Notes |")
        lines.append("|------|------|------|----------|-------|")
        for item in day.get("activities", []):
            time_str = f"{item.get('start_time','')}–{item.get('end_time','')}" if item.get('end_time') else item.get('start_time','')
            lines.append(
                f"| {time_str} | {item.get('activity_type','')} | {item.get('name','')} | {item.get('location','')} | {item.get('notes','')} |"
            )
        lines.append("")

    state["report_markdown"] = "\n".join(lines)
    return state

In [207]:
workflow = StateGraph(State)

# Nodes
workflow.add_node("flight_data", flight_data_node)
workflow.add_node("daily_schedule", daily_schedule_node)
workflow.add_node("report", report_node)

# Edges
workflow.add_edge(START, "flight_data")
workflow.add_edge("flight_data", "daily_schedule")
workflow.add_edge("daily_schedule", "report")
workflow.add_edge("report", END)

# Entry point
workflow.set_entry_point("flight_data")

graph = workflow.compile()

In [208]:
print(graph.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	flight_data(flight_data)
	daily_schedule(daily_schedule)
	report(report)
	__end__([<p>__end__</p>]):::last
	__start__ --> flight_data;
	daily_schedule --> report;
	flight_data --> daily_schedule;
	report --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [209]:
graph.get_graph().print_ascii()

  +-----------+    
  | __start__ |    
  +-----------+    
         *         
         *         
         *         
  +-------------+  
  | flight_data |  
  +-------------+  
         *         
         *         
         *         
+----------------+ 
| daily_schedule | 
+----------------+ 
         *         
         *         
         *         
    +--------+     
    | report |     
    +--------+     
         *         
         *         
         *         
    +---------+    
    | __end__ |    
    +---------+    


### Run Graph

In [210]:
TRIP_START = "2025-03-10"
TRIP_END = "2025-03-15"
START_FLIGHT = "UO870"
RETURN_FLIGHT = "UO871"
PREFERENCES = {
    "pace": "moderate",
    "interests": ["food", "culture", "light hiking"],
    "budget": "mid",
}

#EMAIL_ADDRESS = "terence2379@gmail.com"

In [211]:
input_message = HumanMessage(
    content=(
        "Plan a 5-day trip with start flight is {START_FLIGHT} and return flight is {RETURN_FLIGHT}"
    )
)

In [212]:
events = graph.stream(
    input={
        "messages": [input_message],
        "trip_start_date": TRIP_START,
        "trip_end_date": TRIP_END,
        "start_flight": START_FLIGHT,
        "return_flight": RETURN_FLIGHT,
        "traveler_preferences": PREFERENCES,
    },
    config={"recursion_limit": 50},  # Increased from 30
    stream_mode="values",
)
final_report = None
for event in events:
    if "report_markdown" in event:
        final_report = event["report_markdown"]

if final_report:
    display(Markdown(final_report))
else:
    print("No final report produced. Check inputs or graph configuration.")

# Trip Plan: 2025-03-10 to 2025-03-15

## Overview
- **City:** 
- **Preferences:** {'pace': 'moderate', 'interests': ['food', 'culture', 'light hiking'], 'budget': 'mid'}
- **Flights:**
  - Outbound: Hong Kong International Airport → Tokyo Narita International Airport (10:46–14:55)
  - Return: Tokyo Narita International Airport → Hong Kong International Airport (16:49–21:03)

## Daily Schedule
### Day1
| Time | Type | Name | Location | Notes |
|------|------|------|----------|-------|
| 10:46–14:55 | transit | Flight from Hong Kong to Tokyo | Tokyo Narita International Airport | International flight arrival |
| 15:15–16:15 | transit | Airport to Hotel Transfer | Tokyo city center | Narita Express train to central Tokyo |
| 16:15–17:30 | rest | Check-in and Rest | Hotel | Settle in and refresh after travel |
| 18:00–19:00 | food | Ramen Dinner at Ichiran | Shibuya | Famous tonkotsu ramen chain, individual booths |
| 19:15–20:00 | light_hiking | Evening Walk at Shibuya Crossing | Shibuya | Experience the famous scramble crossing and Hachiko statue |

### Day2
| Time | Type | Name | Location | Notes |
|------|------|------|----------|-------|
| 09:00–11:00 | food | Tsukiji Outer Market Exploration | Tsukiji | Sample fresh seafood, tamagoyaki, and street food |
| 11:15–12:45 | light_hiking | Hama-rikyu Gardens | Chuo Ward | Traditional Japanese landscape garden with tea house |
| 13:00–14:00 | meal | Sushi Lunch at Local Restaurant | Ginza | Mid-range sushi restaurant in Ginza area |
| 14:30–17:00 | culture | Tokyo National Museum | Ueno Park | Largest art museum in Japan, extensive samurai collection |
| 18:00–19:00 | food | Yakitori Dinner at Memory Lane | Shinjuku | Traditional yakitori alley with small restaurants |

### Day3
| Time | Type | Name | Location | Notes |
|------|------|------|----------|-------|
| 09:00–10:30 | culture | Meiji Shrine Visit | Shibuya | Shinto shrine dedicated to Emperor Meiji, peaceful forest setting |
| 10:45–12:00 | light_hiking | Yoyogi Park Walk | Shibuya | Large urban park adjacent to Meiji Shrine |
| 12:30–13:30 | meal | Takoyaki Lunch at Harajuku | Harajuku | Street food experience in Takeshita Street |
| 14:15–16:30 | culture | Senso-ji Temple | Asakusa | Tokyo's oldest temple, Nakamise shopping street |
| 18:00–19:00 | food | Tempura Dinner at Tenichi | Ginza | Established tempura restaurant with counter seating |

### Day4
| Time | Type | Name | Location | Notes |
|------|------|------|----------|-------|
| 09:00–13:00 | light_hiking | Mount Takao Hike | Hachioji | Easy mountain trail with cable car option, temple at summit |
| 13:15–14:15 | meal | Soba Lunch at Mountain Restaurant | Mount Takao | Traditional buckwheat noodles at mountain base |
| 15:30–17:00 | culture | Tokyo Metropolitan Government Building | Shinjuku | Free observation decks with panoramic city views |
| 18:00–19:00 | food | Okonomiyaki Dinner at Sometaro | Asakusa | Traditional Japanese savory pancake cooked at table |

### Day5
| Time | Type | Name | Location | Notes |
|------|------|------|----------|-------|
| 09:00–11:00 | light_hiking | Imperial Palace East Gardens | Chiyoda | Former castle grounds with beautiful seasonal gardens |
| 11:30–13:30 | culture | Edo-Tokyo Museum | Ryogoku | History of Tokyo from Edo period to present |
| 13:45–14:45 | meal | Chanko Nabe Lunch | Ryogoku | Sumo wrestler's hot pot in sumo district |
| 15:30–17:30 | food | Japanese Cooking Class | Shinjuku | Learn to make sushi, tempura, and miso soup |
| 18:00–19:30 | food | Izakaya Dinner Experience | Shinjuku | Japanese pub-style dining with small plates and drinks |

### Day6
| Time | Type | Name | Location | Notes |
|------|------|------|----------|-------|
| 09:00–10:30 | light_hiking | Ueno Park Morning Walk | Ueno | Large public park with temples, museums, and zoo |
| 10:45–12:15 | food | Ameyoko Market Food Tour | Ueno | Bustling market street with diverse food stalls |
| 12:45–14:00 | culture | Last Minute Souvenir Shopping | Ginza | Department stores and specialty shops for gifts |
| 14:30–15:45 | transit | Airport Transfer | Tokyo Narita International Airport | Narita Express to airport for return flight |
| 16:49–21:03 | transit | Flight to Hong Kong | Tokyo Narita International Airport | Return flight departure |
